In [ ]:
# Importing Required Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Reading Data from provided .csv file
data = pd.read_csv('../input/digit-recognizer/train.csv')
print(data.info())
data.head()

In [ ]:
# Retrieving labels from Data
labels = data['label']

# Converting into One-Hot Vectors for Classification
labels = pd.get_dummies(labels)
labels.shape

In [ ]:
# Retrieving all pixel columns from Data
data = data.iloc[:,1:]
data.shape 

In [ ]:
# Taking 40000 images to Train the model
train_data = data[2000:]
train_labels = labels[2000:]

train_data.shape, train_labels.shape

In [ ]:
# Taking 2000 images for Cross Validation
test_data = data[:2000]
test_labels = labels[:2000]

test_data.shape, test_labels.shape

In [ ]:
# Standard loc and scale values for kernel initializer 
def initialize_weights(shape, dtype=None):
    
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

# Standard loc and scale values for bias initializer
def initialize_bias(shape, dtype=None):
    
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

def DeepLearningModel(input_shape):
    model = Sequential()
    
    model.add(Input(input_shape))
    
    model.add(Dense(64, activation='relu', kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(3e-3)))
    
    model.add(Dense(128, activation='relu', kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(3e-3)))

    model.add(Dense(256, activation='relu', kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(3e-3)))
                
    model.add(Dense(10, activation='softmax', kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(3e-3)))    
    
    return model


model = DeepLearningModel((784))
model.summary()

In [ ]:
# Hyperparameters
lr = 0.0003
epochs = 50
batch_size = 128

optimizer = Adam(lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size)

In [ ]:
# Evaluating model using Validation Data
model.evaluate(test_data, test_labels)

In [ ]:
# Reading Test Dataset
test = pd.read_csv('../input/digit-recognizer/test.csv')
test.info()

In [ ]:
# Making Predictions on existing model.
prediction = model.predict(test)

# Rounding the prediction and converting float64 output to int64 output.
prediction = np.array(np.round(prediction), dtype='int64')

# Converting one-hot encoding into label encoding
prediction = (np.argmax(prediction, axis=1)).reshape(-1, 1)

# Creating a DataFrame similar to Example Submission
out = [{'ImageId': i+1, 'Label': prediction[i][0]} for i in range(len(prediction))]

# Creating a .csv file from the DataFrame
pd.DataFrame(out).to_csv('./submission.csv', index=False)

## Achieved 97% accuracy with Deep Learning Classifier

Now let's see how a Convolutional Neural Network will perform on the Dataset.

In [ ]:
imgdata = np.reshape(np.array(data),(data.shape[0],28,28,1))
imglabels = labels

imgdata.shape, imglabels.shape

In [ ]:
# Taking 40000 images to Train the model
img_train_data = imgdata[2000:]
img_train_labels = imglabels[2000:]

img_train_data.shape, img_train_labels.shape

In [ ]:
# Taking 2000 images for Cross Validation
img_test_data = imgdata[:2000]
img_test_labels = imglabels[:2000]

img_test_data.shape, img_test_labels.shape

In [ ]:
# Plotting Dataset

plt.figure(figsize=(6, 6))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img_train_data[i])
    plt.tight_layout()
plt.show()

In [ ]:
# Building an Image Dataset for a Convolutional Neural Network.
def convolutional_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    
    model.add(MaxPooling2D())
    
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    
    model.add(MaxPooling2D())
    
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    model.add(Dense(10, activation='softmax',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    return model

In [ ]:
model = convolutional_model((28,28,1))
model.summary()

In [ ]:
# Hyperparameters
lr = 0.0003
epochs = 10
batch_size = 128

optimizer = Adam(lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(img_train_data, img_train_labels, epochs=epochs, batch_size=batch_size)

In [ ]:
model.evaluate(img_test_data, img_test_labels)

In [ ]:
# Reading Test Dataset
test = pd.read_csv('../input/digit-recognizer/test.csv')
test = np.reshape(np.array(test), (test.shape[0], 28, 28, 1))

In [ ]:
# Making Predictions on existing model.
prediction = model.predict(test)

# Rounding the prediction and converting float64 output to int64 output.
prediction = np.array(np.round(prediction), dtype='int64')

# Converting one-hot encoding into label encoding
prediction = (np.argmax(prediction, axis=1)).reshape(-1, 1)

# Creating a DataFrame similar to Example Submission
out = [{'ImageId': i+1, 'Label': prediction[i][0]} for i in range(len(prediction))]

# Creating a .csv file from the DataFrame
pd.DataFrame(out).to_csv('submission_v2.csv', index=False)